Movie reviews classification using Deep Learning-Sentiment Analysis


In [1]:
from tensorflow.python.keras.datasets import imdb
(x_train,y_train),(x_test,y_tests) = imdb.load_data(num_words = 10000)#the bag of words is 10k which is the vocabulary
print(len(x_train))

25000


In [2]:
print(x_train[0])
res = []
for i in x_train:
    res.append(max(i))
print(max(res))

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
9999


In [3]:
class_names = ['Negative','Positive']#label->0 rep. -tive review while label 1 ->rep pos review

In [4]:
word_index = imdb.get_word_index()#this is a dictionary with word-token pairs
print(word_index['hello'])

4822


In [5]:
token_to_word = dict()
for key,value in word_index.items():
    token_to_word.update({value:key})
    
def decode(review):
    text = ''
    for i in review:
        text = text+token_to_word[i]
        text+=" "
    return text

In [6]:
print(decode(x_train[0]))#the order of the text is not maintained as its a dictionary

the as you with out themselves powerful lets loves their becomes reaching had journalist of lot from anyone to have after out atmosphere never more room and it so heart shows to years of every never going and help moments or of every chest visual movie except her was several of enough more with is now current film as you of mine potentially unfortunately of you than him that with out themselves her get for was camp of you movie sometimes movie that with scary but and to story wonderful that in seeing in character to of 70s musicians with heart had shadows they of here that with her serious to have does when from why what have critics they is you that isn't one will very to as itself with other and in of seen over landed for anyone of and br show's to whether from than out themselves history he name half some br of and odd was two most of mean for 1 any an boat she he should is thought frog but of script you not while history he heart to real at barrel but when from one bit then have tw

In [7]:
def show_len(review):
    print(len(review))
show_len(x_train[0])
show_len(x_train[1])
show_len(x_test[0])
show_len(x_test[1])


218
189
68
260


In [8]:
word_index['the']
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train = pad_sequences(x_train, value = word_index['the'], padding = 'post', maxlen = 256)
x_test = pad_sequences(x_test, value = word_index['the'], padding = 'post', maxlen = 256)

In [9]:
show_len(x_train[0])
decode(x_train[0])

256


"the as you with out themselves powerful lets loves their becomes reaching had journalist of lot from anyone to have after out atmosphere never more room and it so heart shows to years of every never going and help moments or of every chest visual movie except her was several of enough more with is now current film as you of mine potentially unfortunately of you than him that with out themselves her get for was camp of you movie sometimes movie that with scary but and to story wonderful that in seeing in character to of 70s musicians with heart had shadows they of here that with her serious to have does when from why what have critics they is you that isn't one will very to as itself with other and in of seen over landed for anyone of and br show's to whether from than out themselves history he name half some br of and odd was two most of mean for 1 any an boat she he should is thought frog but of script you not while history he heart to real at barrel but when from one bit then have t

In [10]:

from tensorflow.python.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,GlobalAveragePooling1D,Dropout
from keras import regularizers
model = Sequential(
[Embedding(10000,16),#embedding layer which gives 16 feature per word for the 10k words 
GlobalAveragePooling1D(),
Dense(16,kernel_regularizer = regularizers.l2(0.001),activation = "relu"),#16 nodes with l2 weight regularizers to force weights to take only small values.
Dropout(0.5),
Dense(1,activation = "sigmoid")]
)

model.compile(loss = "binary_crossentropy",optimizer = "adam",metrics = ['accuracy'])
model.summary()


Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dropout (Dropout)            (None, 16)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 160,289
Trainable params: 160,289
Non-trainable params: 0
_________________________________________________________________


In [11]:
from tensorflow.python.keras.callbacks import LambdaCallback
simple_log = LambdaCallback(on_epoch_end = lambda e,l : print(e,end = '.'))#on completion of each output print the epoch,log and a ".
epoch = 20
h = model.fit(x_train,y_train,validation_split = 0.2,epochs = epoch,callbacks = [simple_log],verbose = False)

C:\Users\dynam\anaconda3\envs\MyEnv\lib\site-packages\tensorflow\python\ops\gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


0.1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.

In [31]:
def custom_review(custom_str):
    custom_split = custom_str.split(" ")
    custom_encode = [[word_index[x] for x in custom_split]]
    custom_padded = pad_sequences(custom_encode,value = word_index['the'],padding = 'post',maxlen = 256)
    custom_pred = model.predict(custom_padded)
    print(custom_pred)
    print(class_names[0 if custom_pred<(0.36) else 1])


In [32]:
import numpy as np

In [33]:
custom_review("an okay movie that gives importance to shared universe setup over developing a coherent plot")

[[0.33301252]]
Negative


In [34]:
custom_review("good movie i liked it")

[[0.42936707]]
Positive


In [35]:
custom_review("nice movie")

[[0.39183736]]
Positive


In [36]:
custom_review("rubbish movie")

[[0.3315256]]
Negative


In [37]:
custom_review("not bad movie")

[[0.36165968]]
Positive


In [38]:
custom_review("bad movie")

[[0.35816902]]
Negative


In [39]:
np.mean(h.history['loss'])

0.292988935803175

In [40]:
custom_review("too good movie")

[[0.40570828]]
Positive
